In [13]:
import tkinter as tk
from dragonfly import maximise_function, minimise_function
import socket
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
import re

N = 3

server_address_probecampost = ('192.168.15.24', 65280)
server_address_probecampostnear = ('192.168.15.25', 65264)
hexapod_address = ('192.168.15.16', 65529)

subcriptionstring = b'Wait>>MeanCounts'
SubcriptionCmdLength = len(subcriptionstring)
sizepack = struct.pack('>i', SubcriptionCmdLength)

subcriptionstring2 = b'Wait>>Centroidx,Centroidy'
SubcriptionCmdLength2 = len(subcriptionstring2)
sizepack2 = struct.pack('>i', SubcriptionCmdLength2)

In [14]:
def get_centroids():
    cen_cam1_x = entry_cen_x_cam1.get()
    cen_cam1_y = entry_cen_y_cam1.get()
    cen_cam2_x = entry_cen_x_cam2.get()
    cen_cam2_y = entry_cen_y_cam2.get()
    
def get_cam_centroid(cam_server_adress): #select camera
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(cam_server_adress)
    client.sendall(sizepack2 + subcriptionstring2)
    
    #initialise loop
    centroid = np.zeros([N,2])
    i = 0
    
    while i < N:
        size = struct.unpack('>i', client.recv(4))[0]  # Extract the msg size from four bytes - mind the encoding
        str_data = client.recv(size)

        data = re.findall(r'\d+', str_data.decode('ascii'))
        #print(data)
        centroid[i,0] = np.int(data[2])
        centroid[i,1] = np.int(data[4])
        #print('Data size: %s Data value: %s' % (size, str_data.decode('ascii')))
        i = i+1
    client.close()
    
    cenx = np.mean(centroid[:,0]) 
    ceny = np.mean(centroid[:,1]) 
    
    return [cenx,ceny]
    
def initialise():

    movehexapodtooutpos = b"setypos>>20.000"
    udp_client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    udp_client.sendto(movehexapodtooutpos, hexapod_address)
    time.sleep(30)
    
    cenx1,ceny1 = get_cam_centroid(server_address_probecampost)
    cenx2,ceny2 = get_cam_centroid(server_address_probecampostnear)         
    
    hexy.insert(0,cenx1)
    print('done')
    return [cenx1,ceny1,cenx2,ceny2]


In [15]:
master = tk.Tk()

#Close GUI Button
button_close = tk.Button(master, text="Close", command=master.destroy,bg='pink')
button_close.place(x=400,y=100)

Initialise = tk.Button(master, text="Initialise",command=initialise, bg='blue')
Initialise.place(x=400,y=30)

PreAlign = tk.Button(master, text="Pre-Alignment",bg='green')
PreAlign.place(x=400,y=50)

#Camera centroid input

tk.Label(text='starting point', relief=tk.RIDGE, width=15).grid(row=0,column=1)
tk.Label(text='range', relief=tk.RIDGE, width=15).grid(row=0,column=2)
tk.Label(text='hex y', relief=tk.RIDGE, width=15).grid(row=1,column=0)
tk.Label(text='hex z', relief=tk.RIDGE, width=15).grid(row=2,column=0)
tk.Label(text='wangle', relief=tk.RIDGE, width=15).grid(row=3,column=0)
tk.Label(text='vangle', relief=tk.RIDGE, width=15).grid(row=4,column=0)

hexy = tk.Entry(master)
hexy.insert(0,1)
hexy.grid(row=1, column=1)

hexz = tk.Entry(master).grid(row=2, column=1)
wangle = tk.Entry(master).grid(row=3, column=1)
vangle = tk.Entry(master).grid(row=4, column=1)
hexy_r = tk.Entry(master).grid(row=1, column=2)
hexz_r = tk.Entry(master).grid(row=2, column=2)
wangle_r = tk.Entry(master).grid(row=3, column=2)
vangle_r = tk.Entry(master).grid(row=4, column=2)

tk.Label(text='N. Cam:', relief=tk.RIDGE, width=15).grid(row=5,column=0)
Ncamimg = tk.Entry(master).grid(row=5, column=1)

tk.Label(text='Capital:', relief=tk.RIDGE, width=15).grid(row=6,column=0)
Capital = tk.Entry(master).grid(row=6, column=1)


In [16]:
#start GUI
master.geometry("500x170")
tk.mainloop()

done


In [ ]:
#import mainMLcode

#%run mainMLcode.py